In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import sys
import time
import gc
import itertools
import tqdm

from sklearn.preprocessing import LabelEncoder
import scipy.stats as ss

if os.name=='nt':
    try:
        mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
        os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
    except:
        pass
    
import xgboost as xgb
from sklearn.metrics import mean_squared_error 
from numba import jit

%matplotlib inline

### Load data

In [2]:
# Load data
train = pd.read_csv('all/sales_train.csv.gz')
test = pd.read_csv('all/test.csv.gz')
shop = pd.read_csv('all/shops-translated.csv')
item = pd.read_csv('all/item_category.csv')

test.set_index('ID', inplace=True)
item.drop(['item_name_translated'], axis=1, inplace=True)
shop.drop(['Name'], axis=1, inplace=True)

le = LabelEncoder()
item['item_cat1'] = le.fit_transform(item['item_cat1'].astype(str))
item['item_cat2'] = le.fit_transform(item['item_cat2'].astype(str))
shop['City'] = le.fit_transform(shop['City'])
shop['Type'] = le.fit_transform(shop['Type'])

shop.set_index('shop_id', inplace=True)
item.set_index('item_id', inplace=True)

In [3]:
train['sales'] = train['item_price']*train['item_cnt_day']
train = train.merge(shop, how='left', left_on='shop_id', right_index=True)
train = train.merge(item, how='left', left_on='item_id', right_index=True)
train.date = pd.to_datetime(train.date, format='%d.%m.%Y')

test = test.merge(shop, how='left', left_on='shop_id', right_index=True)
test = test.merge(item, how='left', left_on='item_id', right_index=True)

### `cnt`

In [4]:
feature_dict = {} # dict containing all the features

In [5]:
# Total count of sales per month
train_p = train.pivot_table(index=['shop_id', 'item_id'],
                           columns='date_block_num',
                           values='item_cnt_day',
                           aggfunc=np.sum).fillna(0.0)
# Rename columns
train_p.columns = ['cnt_'+str(k) for k in train_p.columns.get_level_values(0)]
# Add one column for test data
train_p['cnt_34'] = np.zeros((len(train_p), 1))
# Merge to test
train_p = test.merge(train_p, how='left', left_on=['shop_id', 'item_id'], right_index=True)
# Add to the feature dict
feature_dict['cnt'] = train_p

### `sales`

In [6]:
# Total value of sales per month
sales = train.pivot_table(index=['shop_id', 'item_id'],
                         columns='date_block_num', 
                         values='sales',
                         aggfunc=np.sum).fillna(0.0)
# Rename columns
sales.columns = ['sales_'+str(k) for k in sales.columns.get_level_values(0)]
# Add on column for test data
sales['sales_34'] = np.zeros((len(sales), 1))
# Merge to test
sales = test.merge(sales, how='left', left_on=['shop_id', 'item_id'], right_index=True)
# Add to the feature dict
feature_dict['sales'] = sales

### explore

In [7]:
# Apply pivot table to x with combinations of the following parameters
index_list = ['item_id', 'shop_id', 'City', 'Type', 'item_cat1', 'item_cat2']
column_list = ['date_block_num']
value_list = ['item_price', 'item_cnt_day', 'sales']
aggfunc_list = [np.sum, np.mean]

In [ ]:
feature_dict2 = {}
for idx, col, val, agf in tqdm.tqdm_notebook(list(itertools.product(index_list, column_list, value_list, aggfunc_list))):
    tmp = train.pivot_table(index=idx, columns=col, values=val, aggfunc=agf).fillna(0.0)
    tmp_name = '-'.join([idx, val, str(agf).split(' ')[1]])
    tmp.columns = [tmp_name+'_'+str(k) for k in tmp.columns]
    tmp[tmp_name+'_34'] = np.zeros((len(tmp), 1))
    feature_dict2[tmp_name] = tmp

### CV